In [4]:

%matplotlib inline


PATH_TO_DATA = '../../data/statefarm/'
PATH_TO_IMGS = PATH_TO_DATA + 'imgs/'
train_data_dir = PATH_TO_IMGS+'train/'
validation_data_dir = PATH_TO_IMGS+'/validation/'
test_data_dir = PATH_TO_IMGS+'/test/'
img_width, img_height = 80,80 #150, 150

labelMapping = { 'c0': 'normal driving', 
                 'c1': 'texting - right',
                'c2': 'talking on the phone - right',
                'c3': 'texting - left',
                'c4': 'talking on the phone - left',
                'c5': 'operating the radio',
                'c6': 'drinking',
                'c7': 'reaching behind',
                'c8': 'hair and makeup',
                'c9': 'talking to passenger' 
               }


In [5]:
from keras.preprocessing.image import ImageDataGenerator #as ImageDataGeneratorBase
import numpy as np



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        shear_range=0.1,
        rotation_range=8,       #degrees#rotation_range: Int. Degree range for random rotations.
        width_shift_range=0.05, #width_shift_range: Float (fraction of total width). Range for random horizontal shifts.
        height_shift_range=0.05,  #height_shift_range: Float (fraction of total height). Range for random vertical shifts.
        zoom_range=0.1,  
        horizontal_flip=False)
train_datagen.mean = np.load('mean_image'+str(img_width)+'.npy')
train_datagen.std= np.load('std_image'+str(img_width)+'.npy')
#this is the augmentation configuration we will use for testing: only rescaling + mean and std of training
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rescale=1./255)
test_datagen.mean = np.load('mean_image'+str(img_width)+'.npy')
test_datagen.std= np.load('std_image'+str(img_width)+'.npy')


train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')  #label infered from folder


validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical') #label infered from folder




def saveToFiles(model,filenameWithoutSuffix):
    ''' will create X.json and X.h5 files'''
    with open(filenameWithoutSuffix +'.json', "w") as json_file:
        json_file.write(model.to_json() )
    model.save_weights(filenameWithoutSuffix +'.h5')
    print("Saved model to disk "+ filenameWithoutSuffix)

def loadFromFiles(filenameWithoutSuffix):
    from keras.models import model_from_json
    json_file = open(filenameWithoutSuffix+ '.json', 'r')
    json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(json)
    loaded_model.load_weights(filenameWithoutSuffix +'.h5')
    return loaded_model
print 'v5'
#saveToFiles(loaded_model2,'model1_21epoc')

Found 18819 images belonging to 10 classes.
Found 3034 images belonging to 10 classes.
v5


In [3]:


def calculat_std_and_mean_images_on_RGB_W_H():

  
    generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_data_dir,
        target_size=(img_width,img_height),
        batch_size=1,
        shuffle=True,
        class_mode=None)
   

    channel_to_mean_list = {'R':[],'G':[],'B':[]}
    channel_to_std_list =  {'R':[],'G':[],'B':[]}
 
    num_of_batches= 100#  10000/batch_size  #there are 20000 for both test and train so let's take subset  
    print 'will only work on',num_of_batches,'batches of them  '
    for i in range(0,num_of_batches):
        if i%200==1:  
            print 'running on #',i
          
        x= next(generator)[0]  #batch is always one!
        for i,channel in enumerate (['R','G','B']):
            channel_to_mean_list[channel].append(np.mean(x[i,:,:]))
            channel_to_std_list[channel].append(np.std((x[i,:,:])))
      
    #create an array of the same shape
    mean_image = np.empty([3,img_width,img_height])
    std_image = np.empty([3,img_width,img_height])
    
    #calcute the mean of all the images together. mean will have 3 values reperated meanR,meanG,meanB
  
    for i,channel in enumerate ( ['R','G','B']):
        mean_channel_list= channel_to_mean_list[channel]
        mean_image[i] = np.mean(mean_channel_list) # mean of 100 values, is just mean
        #print 'mean',mean_image[i][0,0],mean_channel_list
        
        std_channel_list= channel_to_std_list[channel]                                        
        std_image[i] =  np.sqrt( np.sum(np.square(std_channel_list))/float(len(std_channel_list)))
        #print 'std',std_image[i][0,0],std_channel_list
    return mean_image,std_image
   
def calculateAndSaveOnce():  #must be called once at least!!!!
    mean_image,std_image=calculat_std_and_mean_images_on_RGB_W_H() 
    print mean_image[0,0,0] ,mean_image[1,0,0] ,mean_image[2,0,0] # 0.305443632772 0.374382142377 0.364442742236
    print std_image[0,0,0],std_image[1,0,0],std_image[2,0,0]      # 0.284977836867 0.330808235492 0.332097428032
    np.save('mean_image'+str(img_height),mean_image)
    np.save('std_image'+str(img_height),std_image)
    
calculateAndSaveOnce()

Found 18819 images belonging to 10 classes.
will only work on 100 batches of them  
running on # 1
0.311151806362 0.378612648303 0.368988604246
0.288531686315 0.332663278551 0.332838195427


In [ ]:
import os.path
import numpy as np
import pandas
from time import gmtime, strftime


def if_prob_slim_change_it(all_probs):
    ''' all_probs is of shape ROWSx10 '''
    confidanceTracker ={'high':0, 'medium':0, 'low':0}
    for i in range(0,all_probs.shape[0]):
        confidance = np.sum(all_probs[i])  #usually much less than 1
        if np.sum(all_probs[i])<0.3:
            confidanceTracker['low']= 1+ confidanceTracker['low']
            all_probs[i]+=0.1
        elif np.sum(all_probs[i])<0.9:
            confidanceTracker['medium']= 1+ confidanceTracker['medium']
            all_probs[i]+=0.001
        else:
            confidanceTracker['high']= 1+ confidanceTracker['high']
    print confidanceTracker


def predict_all_directory(loaded_modelX,directory,limit_batches=0):
    '''
    limit_batches=0 means no limit(full directory). positive means we will cap after X batches'''
    #let's make sure to recreate it, so generator will re-init
    batch_size=32
    submit_generator = test_datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=False,
        class_mode=None) #no labels    

    num_of_batchs = len(submit_generator.filenames)/batch_size
    mod = len(submit_generator.filenames)%batch_size
    if mod: 
        num_of_batchs+=1
    if (limit_batches>0):
        num_of_batchs=limit_batches
    
    list_of_probs_batches=[]
    for batch_i in range(0,num_of_batchs):
        x_list= next(submit_generator)  #batch of 32 images
        x = np.asarray(x_list)
        curr_probs = loaded_modelX.predict( x, batch_size)
        list_of_probs_batches.append(curr_probs)
        if batch_i%50==1:
            print batch_i, list_of_probs_batches[batch_i].shape
    all_probs =  np.concatenate(list_of_probs_batches,axis=0)
    return submit_generator.filenames[:all_probs.shape[0]] , all_probs

def predict_to_csv(filenameNoPrefix,pathname_list,all_probs):
    
    # The expected out file should be:
    #img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
    #img_0.jpg,1,0,0,0,0,...,0
    #img_1.jpg,0.3,0.1,0.6,0,...,0
    name_list = [os.path.basename(x) for x in pathname_list]
    data_dict = { 'img':name_list }
    for i in range(0,10):
        data_dict['c'+str(i)]= all_probs[:,i]

    my_solution = pandas.DataFrame.from_dict(data_dict)

    # Check that your data frame has 418 entries
    print 'csv shape', my_solution.shape
    print my_solution.describe()
    pandas.set_option('display.width', 800)
    print(my_solution)


    # Write your solution to a csv file with the name my_solution.csv
    filename='state_farm_PE_'+filenameNoPrefix+strftime("_%Y-%m-%d_%H-%M", gmtime())+".csv"
    my_solution.to_csv(filename,
                       columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
                       , index=False,index_label = ["img"])
    print 'save complete to file '+filename
    
    #now cut few rows
    my_solution = my_solution[0:79726]
    filename='state_farm_PE_'+filenameNoPrefix+'_cut79726_'+strftime("_%Y-%m-%d_%H-%M", gmtime())+".csv"
    my_solution.to_csv(filename,
                       columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
                       , index=False,index_label = ["img"])
    print 'save complete to file '+filename
 
    for i in range(0,10):
        my_solution['c'+str(i)] = my_solution['c'+str(i)].map('{:,.8f}'.format)
        filename='state_farm_P8_'+filenameNoPrefix+strftime("_%Y-%m-%d_%H-%M", gmtime())+".csv"
    my_solution.to_csv(filename,
                       columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
                       , index=False,index_label = ["img"])
    print 'save complete to file '+filename
    



In [ ]:

import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

pred = [(0.5,0.5)]      
act = [(1,0)]
#print (logloss((1,0), (0.5,0.5))) #0.69
#print (logloss((0,1), (1,0)))     #very wrong- 34.5
#print (logloss((1,0), (1,0)))     # true - ~0 (9e-16)
#print (logloss((1,0,0,0), (0,1,1,1))) # 0.1

def score_batch(actual_class_list,pred_nparray,verbose=0):
    '''
        return score and misses indexes loss bigger than 0.1 on one item
    '''
    #create an array from one class
    print 'calculating score of' , pred_nparray.shape[0], 'results'
    if len(actual_class_list)!=pred_nparray.shape[0] :
        raise ValueError('wrong shapes')
    scores=[]
    miss_scores=[]
    miss_index=[]
    for i,c in enumerate(actual_class_list):
        act_list = [ 0  for _ in range(0,pred_nparray.shape[1])]
        act_list[int(c)]=1
        
        pred_list = pred_nparray[i].tolist()
        curr_score=logloss(act_list,pred_list )
        scores.append(curr_score)
        if curr_score>0.1:
            miss_index.append(i)
            miss_scores.append(curr_score)
            if verbose:
                print i,'WRONG',curr_score #,act_list,pred_list
        elif verbose:
                print i,'RIGHT\t\t\t',curr_score #,act_list,pred_list
       
    if verbose:
        print 'median'  ,np.median(np.array(scores))
        print 'mean',np.mean(np.array(scores))
        print 'misses >0.1 mean',np.mean(np.array(miss_scores))
        print 'misses >0.1 median',np.median(np.array(miss_scores))
    
    return sum(scores)/pred_nparray.shape[0],miss_index
        


In [ ]:
def calculate_validation_score(model,limit_batches=0,verbose=0):
    pathname_list,all_probs= predict_all_directory(model,validation_data_dir,limit_batches) #0 no limit
    name_list = [os.path.dirname(x)[1] for x in pathname_list]
    score,miss_indexes= score_batch(name_list,all_probs)
    accuracy = 1- float(len(miss_indexes))/ len(name_list)
    if verbose:
        print '\nSCORE=',score,'accuracy=',accuracy
        print miss_indexes , 'of total',len(pathname_list)
        print [pathname_list[i]  for i in miss_indexes]
    return score,accuracy

def create_csv(model,name):
    pathname_list,all_probs= predict_all_directory(model,test_data_dir)
    #if_prob_slim_change_it(all_probs)   
    predict_to_csv(name,pathname_list,all_probs)

if __name__ == "__main__":
    print "running statefarm-predict as main. doing nothing"
else:
    print "running statefarm-predict as import only. use me like this: %run 'statefarm-util.ipynb'   "

In [5]:

import matplotlib.pyplot as plt

def flat(history_list,key):
        return sum([history.history[key] for history in history_list],[])
    
def graph_history(history_list):
    #comibe histories
    #keys= history_list[0].history.keys()

    # red dash r--  'bs' Blue Square  'g^' Green Triangles
    x_epocs= range(0,len(history_list))
    
    plt.plot(x_epocs, flat(history_list,'loss'),'b-',label='loss')
    plt.plot(x_epocs, flat(history_list,'val_loss'),'r-',label='val_loss')
    max_val = max( max(flat(history_list,'loss')), max(flat(history_list,'val_loss')))
    plt.axis([0, len(x_epocs), 0, max_val ])
    plt.legend()
    plt.show() 
 
    plt.plot(x_epocs, flat(history_list,'acc'),'b^--',label='acc')
    plt.plot(x_epocs, flat(history_list,'val_acc'),'r^--',label='val_acc')
    plt.axis([0, len(x_epocs), 0, 1])
    plt.legend()
    plt.show() 

    
    

#combine it to the first one...
def graph_history_3(history_list):
   
    
    #comibe histories

    x_epocs= range(0,len(history_list))

    l1 = plt.plot(x_epocs, flat(history_list,'loss'),'b-',label='loss')
    plt.plot(x_epocs, flat(history_list,'val_loss'),'r-',label='val_loss')
             
    l2 =plt.plot(        x_epocs, flat(history_list,'prob_loss'),'bo--',label='c2')
    plt.plot(         x_epocs, flat(history_list,'val_prob_loss'),'ro--',label='c2')
             
    l3= plt.plot(         x_epocs, flat(history_list,'prob_aux0_loss'),'bs--',label='c0')
    plt.plot(         x_epocs, flat(history_list,'val_prob_aux0_loss'),'rs--',label='c0')
             
    l4=plt.plot(         x_epocs, flat(history_list,'prob_aux1_loss'),'b^--',label='c1')
    plt.plot(         x_epocs, flat(history_list,'val_prob_aux1_loss'),'r^--',label='c1')
            
    plt.axis([0, len(x_epocs), 0, 1])
    plt.legend()
    plt.show()   
    
    plt.plot(
             
             x_epocs, flat(history_list,'prob_acc'),'bo--',
             x_epocs, flat(history_list,'val_prob_acc'),'ro--',
             
             x_epocs, flat(history_list,'prob_aux0_acc'),'bs--',
             x_epocs, flat(history_list,'val_prob_aux0_acc'),'rs--',
             
             x_epocs, flat(history_list,'prob_aux1_acc'),'b^--',
             x_epocs, flat(history_list,'val_prob_aux1_acc'),'r^--',
            )
    
    
           
    plt.axis([0, len(x_epocs), 0.5, 1.0])
    plt.show()          
    

   
    # red dash r--  'bs' Blue Square  'g^' Green Triangles
   
print 'done'       
   
        

done


In [11]:
from time import gmtime, strftime
def time_str(): 
    return strftime("_%Y-%m-%d_%H-%M", gmtime())
